### 라이브러리 import

In [1]:
import torch
from torch import nn

from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Resize, Grayscale
from torch.utils.data import DataLoader

from torch import optim
from torch.nn import functional

import matplotlib.pyplot as plt

In [2]:
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

### AlexNet 모델 구현

In [3]:
class MyAlexNet(nn.Module):
    def __init__(self, number_of_class=1000):
        super(MyAlexNet, self).__init__()
        self.network = nn.Sequential(
            # Conv 1 - 96 kernels of size 11x11
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2),
            
            # Conv 2
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2),
            
            # Conv 3
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            # Conv 4
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            # Conv 5
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=(6*6*256), out_features=4096),
            nn.ReLU(),
            
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            
            nn.Linear(in_features=4096, out_features=number_of_class)
        )
        
    def forward(self, x):
        x = self.network(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

### 학습

In [4]:
def train(device, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f'loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')

In [5]:
def test(device, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loass: {test_loss:>8f}\n')

In [6]:
def train_n_test(device, model, loss_fn, optimizer, train_dataloader, test_dataloader, epochs=5):
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1} \n--------------------------------')
        lr_scheduler.step()
        train(device, train_dataloader, model, loss_fn, optimizer)
        test(device, test_dataloader, model, loss_fn)

### 호출해보기 - 1

In [8]:
dataset = datasets.ImageFolder('./data/flower_photos', 
               transform=Compose([Resize((227, 227)), ToTensor()])
          )

In [9]:
len(dataset), len(dataset)/2

(3670, 1835.0)

In [10]:
training_data, test_data = torch.utils.data.random_split(dataset, [1835, 1835])

In [11]:
training_data, test_data, len(training_data), len(test_data)

(<torch.utils.data.dataset.Subset at 0x1e9bd8f0288>,
 1835,
 1835)

In [12]:
# BATCH_SIZE = 128
# train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)

train_dataloader = DataLoader(training_data)
test_dataloader = DataLoader(test_data)

In [13]:
len(test_dataloader)

1835

In [14]:
for X, y in test_dataloader:
    print('Shape of X[N, C, H, W]:', X.shape)
    print('Shape of y:', y.shape, y.dtype)
    break

Shape of X[N, C, H, W]: torch.Size([1, 3, 227, 227])
Shape of y: torch.Size([1]) torch.int64


In [15]:
model = MyAlexNet()
model

MyAlexNet(
  (network): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=True)
    (1): Linear(in_features=9216, out_f

In [16]:
LR_INIT = 0.01
MOMENTUM = 0.9
LR_DECAY = 0.0005

optimizer = optim.SGD(
    params=model.parameters(),
    lr=LR_INIT,
    momentum=MOMENTUM,
    weight_decay=LR_DECAY,
)
loss_fn = functional.cross_entropy

In [17]:
device = get_device()
train_n_test(device, model, loss_fn, optimizer, train_dataloader, test_dataloader, 3)

Epoch 1 
--------------------------------


d:\program\python\python37\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
d:\program\python\python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss: 6.902305   [    0/ 1835]
loss: 2.060395   [  100/ 1835]
loss: 2.480644   [  200/ 1835]
loss: 0.382271   [  300/ 1835]
loss: 4.816805   [  400/ 1835]
loss: 10.492311   [  500/ 1835]
loss:     nan   [  600/ 1835]
loss:     nan   [  700/ 1835]
loss:     nan   [  800/ 1835]
loss:     nan   [  900/ 1835]
loss:     nan   [ 1000/ 1835]
loss:     nan   [ 1100/ 1835]
loss:     nan   [ 1200/ 1835]
loss:     nan   [ 1300/ 1835]
loss:     nan   [ 1400/ 1835]
loss:     nan   [ 1500/ 1835]
loss:     nan   [ 1600/ 1835]
loss:     nan   [ 1700/ 1835]
loss:     nan   [ 1800/ 1835]
Test Error: 
 Accuracy: 17.6%, Avg loass:      nan

Epoch 2 
--------------------------------
loss:     nan   [    0/ 1835]
loss:     nan   [  100/ 1835]
loss:     nan   [  200/ 1835]
loss:     nan   [  300/ 1835]
loss:     nan   [  400/ 1835]
loss:     nan   [  500/ 1835]
loss:     nan   [  600/ 1835]
loss:     nan   [  700/ 1835]
loss:     nan   [  800/ 1835]
loss:     nan   [  900/ 1835]
loss:     nan   [ 1000/ 1835]

### 호출해보기 - 2

In [38]:
training_data = datasets.MNIST(
    root='data', train=True, download=True,
    transform=Compose([
        Grayscale(num_output_channels=3),
        Resize((227, 227)),
        ToTensor(),
    ])
)
print('original training_data size:', len(training_data))

subset_length = 5000 + 1
training_data = torch.utils.data.Subset(training_data, [i for i in range(0, subset_length)])
print('subset training_data size:', len(training_data))

original training_data size: 60000
subset training_data size: 5001


In [39]:
test_data = datasets.MNIST(
    root='data', train=False, download=True,
    transform=Compose([
        Grayscale(num_output_channels=3),
        Resize((227, 227)),
        ToTensor(),
    ])
)
print('original test_data size:', len(test_data))

test_data = torch.utils.data.Subset(test_data, [i for i in range(0, subset_length)])
print('subset training_data size:', len(test_data))

original test_data size: 10000
subset training_data size: 5001


In [40]:
train_dataloader = DataLoader(training_data)
test_dataloader = DataLoader(test_data)

In [41]:
model2 = MyAlexNet()
model2

MyAlexNet(
  (network): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=True)
    (1): Linear(in_features=9216, out_f

In [42]:
LR_INIT = 0.01
MOMENTUM = 0.9
LR_DECAY = 0.0005

optimizer = optim.SGD(
    params=model.parameters(),
    lr=LR_INIT,
    momentum=MOMENTUM,
    weight_decay=LR_DECAY,
)
loss_fn = functional.cross_entropy

In [43]:
train_n_test(device, model2, loss_fn, optimizer, train_dataloader, test_dataloader, 1000)

Epoch 1 
--------------------------------
loss: 6.917681   [    0/ 5001]
loss: 6.922639   [  100/ 5001]
loss: 6.921749   [  200/ 5001]
loss: 6.904881   [  300/ 5001]
loss: 6.911610   [  400/ 5001]
loss: 6.908261   [  500/ 5001]
loss: 6.909052   [  600/ 5001]
loss: 6.897280   [  700/ 5001]
loss: 6.906073   [  800/ 5001]
loss: 6.911418   [  900/ 5001]
loss: 6.901933   [ 1000/ 5001]
loss: 6.900286   [ 1100/ 5001]
loss: 6.923900   [ 1200/ 5001]
loss: 6.901096   [ 1300/ 5001]
loss: 6.904312   [ 1400/ 5001]
loss: 6.911658   [ 1500/ 5001]
loss: 6.903975   [ 1600/ 5001]
loss: 6.919896   [ 1700/ 5001]
loss: 6.909002   [ 1800/ 5001]
loss: 6.908118   [ 1900/ 5001]
loss: 6.920502   [ 2000/ 5001]
loss: 6.904340   [ 2100/ 5001]
loss: 6.906506   [ 2200/ 5001]
loss: 6.895329   [ 2300/ 5001]
loss: 6.902744   [ 2400/ 5001]
loss: 6.907590   [ 2500/ 5001]
loss: 6.908937   [ 2600/ 5001]
loss: 6.909175   [ 2700/ 5001]
loss: 6.899239   [ 2800/ 5001]
loss: 6.923913   [ 2900/ 5001]
loss: 6.907804   [ 3000/ 500

KeyboardInterrupt: 

### 호출해보기 - 3

In [28]:
training_data = datasets.STL10(
    root='data', split='train', download=True,
    transform=Compose([
        Resize((227, 227)),
        ToTensor(),
    ])
)
print('original training_data size:', len(training_data))


Extracting data\stl10_binary.tar.gz to data
original training_data size: 5000


In [29]:
test_data = datasets.STL10(
    root='data', split='test', download=True,
    transform=Compose([
        Resize((227, 227)),
        ToTensor(),
    ])
)
print('original test_data size:', len(test_data))

Files already downloaded and verified
original test_data size: 8000


In [34]:
train_dataloader = DataLoader(training_data)
test_dataloader = DataLoader(test_data)

In [35]:
model3 = MyAlexNet()

In [36]:
LR_INIT = 0.01
MOMENTUM = 0.9
LR_DECAY = 0.0005

optimizer = optim.SGD(
    params=model.parameters(),
    lr=LR_INIT,
    momentum=MOMENTUM,
    weight_decay=LR_DECAY,
)
loss_fn = functional.cross_entropy

In [37]:
train_n_test(device, model3, loss_fn, optimizer, train_dataloader, test_dataloader, 1000)

Epoch 1 
--------------------------------
loss: 6.912450   [    0/ 5000]
loss: 6.900013   [  100/ 5000]
loss: 6.916574   [  200/ 5000]
loss: 6.911217   [  300/ 5000]
loss: 6.897245   [  400/ 5000]
loss: 6.903224   [  500/ 5000]
loss: 6.903133   [  600/ 5000]
loss: 6.891865   [  700/ 5000]
loss: 6.912898   [  800/ 5000]
loss: 6.908477   [  900/ 5000]
loss: 6.895142   [ 1000/ 5000]
loss: 6.912224   [ 1100/ 5000]
loss: 6.912503   [ 1200/ 5000]
loss: 6.910120   [ 1300/ 5000]
loss: 6.893977   [ 1400/ 5000]
loss: 6.908743   [ 1500/ 5000]
loss: 6.902219   [ 1600/ 5000]
loss: 6.899412   [ 1700/ 5000]
loss: 6.912559   [ 1800/ 5000]
loss: 6.914780   [ 1900/ 5000]
loss: 6.917825   [ 2000/ 5000]
loss: 6.914239   [ 2100/ 5000]
loss: 6.912888   [ 2200/ 5000]
loss: 6.894862   [ 2300/ 5000]
loss: 6.895098   [ 2400/ 5000]
loss: 6.896302   [ 2500/ 5000]
loss: 6.909850   [ 2600/ 5000]
loss: 6.911009   [ 2700/ 5000]
loss: 6.891070   [ 2800/ 5000]
loss: 6.893897   [ 2900/ 5000]
loss: 6.897862   [ 3000/ 500

KeyboardInterrupt: 

### 호출해보기 - 4

In [48]:
import torchvision
model4 = torchvision.models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\samsung/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


In [51]:
training_data = datasets.MNIST(
    root='data', train=True, download=True,
    transform=Compose([
        Grayscale(num_output_channels=3),
        Resize((227, 227)),
        ToTensor(),
    ])
)
print('original training_data size:', len(training_data))

subset_length = 5000 + 1
training_data = torch.utils.data.Subset(training_data, [i for i in range(0, subset_length)])
print('subset training_data size:', len(training_data))

original training_data size: 60000
subset training_data size: 5001


In [53]:
test_data = datasets.MNIST(
    root='data', train=False, download=True,
    transform=Compose([
        Grayscale(num_output_channels=3),
        Resize((227, 227)),
        ToTensor(),
    ])
)
print('original test_data size:', len(test_data))

test_data = torch.utils.data.Subset(test_data, [i for i in range(0, subset_length)])
print('subset training_data size:', len(test_data))

original test_data size: 10000
subset training_data size: 5001


In [54]:
train_dataloader = DataLoader(training_data)
test_dataloader = DataLoader(test_data)

In [55]:
LR_INIT = 0.01
MOMENTUM = 0.9
LR_DECAY = 0.0005

optimizer = optim.SGD(
    params=model.parameters(),
    lr=LR_INIT,
    momentum=MOMENTUM,
    weight_decay=LR_DECAY,
)
loss_fn = functional.cross_entropy

In [56]:
train_n_test(device, model4, loss_fn, optimizer, train_dataloader, test_dataloader, 5)

Epoch 1 
--------------------------------


d:\program\python\python37\lib\site-packages\torch\optim\lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


loss: 7.496165   [    0/ 5001]
loss: 11.379951   [  100/ 5001]
loss: 8.663388   [  200/ 5001]
loss: 16.175499   [  300/ 5001]
loss: 10.071582   [  400/ 5001]
loss: 10.409053   [  500/ 5001]
loss: 10.951009   [  600/ 5001]
loss: 9.999689   [  700/ 5001]
loss: 12.091156   [  800/ 5001]
loss: 12.328613   [  900/ 5001]
loss: 11.744202   [ 1000/ 5001]
loss: 10.857874   [ 1100/ 5001]
loss: 9.166437   [ 1200/ 5001]
loss: 10.652464   [ 1300/ 5001]
loss: 12.268744   [ 1400/ 5001]
loss: 12.374990   [ 1500/ 5001]
loss: 13.641829   [ 1600/ 5001]
loss: 8.571790   [ 1700/ 5001]
loss: 12.617880   [ 1800/ 5001]
loss: 12.878156   [ 1900/ 5001]
loss: 10.088725   [ 2000/ 5001]
loss: 12.809055   [ 2100/ 5001]
loss: 11.815282   [ 2200/ 5001]
loss: 13.016496   [ 2300/ 5001]
loss: 14.303643   [ 2400/ 5001]
loss: 14.133695   [ 2500/ 5001]
loss: 10.448414   [ 2600/ 5001]
loss: 11.263763   [ 2700/ 5001]
loss: 9.851080   [ 2800/ 5001]
loss: 9.761324   [ 2900/ 5001]
loss: 13.061285   [ 3000/ 5001]
loss: 10.172316

loss: 9.671409   [ 4300/ 5001]
loss: 11.643952   [ 4400/ 5001]
loss: 14.956113   [ 4500/ 5001]
loss: 11.528555   [ 4600/ 5001]
loss: 12.035089   [ 4700/ 5001]
loss: 15.699138   [ 4800/ 5001]
loss: 7.553170   [ 4900/ 5001]
loss: 12.864435   [ 5000/ 5001]
Test Error: 
 Accuracy: 0.0%, Avg loass: 11.623820

